In [1]:
import os

os.environ['OPENAI_API_KEY'] = '****'
os.environ['LANGCHAIN_API_KEY'] = '****'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = '03-04'

In [4]:
!pip install -qU langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [6]:
model = ChatOpenAI(
    temperature=0,
    model = "gpt-4o"
)

In [7]:
class Topic(BaseModel):
  description: str = Field(description="주제에 대한 간결한 설명")
  hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [8]:
parser = JsonOutputParser(pydantic_object=Topic)

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user","#Format:{format_instructions}\n\n#Question:{question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [23]:
chain = prompt | model | parser

response = chain.invoke({"question":"EPL19-20 시즌 토트넘과 아스날 경기에서 가장 활약한 선수 알려주고 어떤 부분에서 활약했는지 설명해"})

print(response)

{'description': 'EPL 19-20 시즌 토트넘과 아스날 경기에서 손흥민이 가장 활약한 선수로 꼽혔습니다. 그는 경기에서 골을 기록하며 팀의 승리에 기여했습니다.', 'hashtags': '#EPL #Tottenham #Arsenal #SonHeungMin'}


## Pydantic 없이 사용

In [44]:
# 질의
question = "UCL 18-19시즌 손흥민의 활약에 대해 알려주세요.  이 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."


# JSON 출력 파서 초기화
parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

response

{'description': '손흥민은 2018-19 UEFA 챔피언스리그에서 토트넘 홋스퍼의 핵심 선수로 활약했습니다. 그는 맨체스터 시티와의 8강전에서 두 경기 합계 3골을 기록하며 팀의 준결승 진출에 크게 기여했습니다. 특히, 에티하드 스타디움에서 열린 2차전에서 두 골을 넣어 팀의 승리를 이끌었습니다. 또한, 아약스와의 준결승 2차전에서도 중요한 역할을 하며 팀의 결승 진출에 기여했습니다.',
 'hashtags': ['#손흥민', '#UCL', '#토트넘', '#챔피언스리그', '#축구']}